In [110]:
#librarys 
import json
import requests
import numpy as np
import pandas as pd
import plotly.express as px
from requests_cache import CachedSession
from datetime import datetime

In [111]:
api_key = '4a8b15e8da98bcfed048039b4851248d'


In [112]:
def get_zip_weather(api_key,zipp, country = 'US'):
    """Get the weather for zip."""
    weather_url = f"https://api.openweathermap.org/data/2.5/weather?zip={zipp},{country}&appid={api_key}&units=imperial"
    response = requests.get(weather_url)
    return response.json()

In [113]:
get_zip_weather(api_key,96729)

{'coord': {'lon': -157.0791, 'lat': 21.173},
 'weather': [{'id': 801,
   'main': 'Clouds',
   'description': 'few clouds',
   'icon': '02d'}],
 'base': 'stations',
 'main': {'temp': 85.46,
  'feels_like': 89.44,
  'temp_min': 79.54,
  'temp_max': 85.46,
  'pressure': 1016,
  'humidity': 58},
 'visibility': 10000,
 'wind': {'speed': 3.44, 'deg': 0},
 'clouds': {'all': 20},
 'dt': 1702338775,
 'sys': {'type': 1,
  'id': 7870,
  'country': 'US',
  'sunrise': 1702313739,
  'sunset': 1702352865},
 'timezone': -36000,
 'id': 0,
 'name': 'Hoolehua',
 'cod': 200}

In [114]:
def State_info(state):
    url = f'https://data.opendatasoft.com/api/explore/v2.1/catalog/datasets/georef-united-states-of-america-zc-point@public/records?limit=100&refine=stusps_code%3A%22{state}%22'
    session = CachedSession()
    response = session.get(url)
    return response.json()

In [115]:
State_info('DE')

{'total_count': 67,
 'results': [{'zip_code': '19944',
   'usps_city': 'Fenwick Island',
   'stusps_code': 'DE',
   'ste_name': 'Delaware',
   'zcta': 'TRUE',
   'parent_zcta': None,
   'population': 697.0,
   'density': 212.2,
   'primary_coty_code': '10005',
   'primary_coty_name': 'Sussex',
   'county_weights': '{"10005": "100"}',
   'coty_name': ['Sussex'],
   'cty_code': ['10005'],
   'imprecise': 'FALSE',
   'military': 'FALSE',
   'timezone': 'America/New_York',
   'geo_point_2d': {'lon': -75.05559, 'lat': 38.47578}},
  {'zip_code': '19709',
   'usps_city': 'Middletown',
   'stusps_code': 'DE',
   'ste_name': 'Delaware',
   'zcta': 'TRUE',
   'parent_zcta': None,
   'population': 42480.0,
   'density': 205.9,
   'primary_coty_code': '10003',
   'primary_coty_name': 'New Castle',
   'county_weights': '{"10003": "100"}',
   'coty_name': ['New Castle'],
   'cty_code': ['10003'],
   'imprecise': 'FALSE',
   'military': 'FALSE',
   'timezone': 'America/New_York',
   'geo_point_2d': {

### Current Weather

In [116]:
def get_weather_map():
    api_key = '4a8b15e8da98bcfed048039b4851248d'
    try:
        state = str(input("Please enter the USPS state code (ie. California is CA): "))
        stateinfo = State_info(state)
        dfstateinfo = pd.DataFrame(stateinfo['results'])
    except ValueError:
        print("Invalid input. Please enter a valid state name. It is cap sensitive.")
    if stateinfo['total_count'] == 0:
        print("Invalid input. Please enter a valid state name. It is cap sensitive.")
    # county info     
    print(set(dfstateinfo['primary_coty_name']))    
    county = str(input(f'Please choose the county in {state} you would like to see from the list above:'))
    zipcodes = dfstateinfo['zip_code'][dfstateinfo['primary_coty_name'] == county]
    
    combined = []
    for zipcode in zipcodes:
        test_data = get_zip_weather(api_key,zipcode)
        #gather subseted data
        weather_info = test_data['weather'][0]
        main_info = test_data['main']
        wind_info = test_data['wind']
        loc_info = test_data['coord']
        city_info = test_data['name']
        # join all to a dictionary 
        sub = {**{'name':city_info},
               **{'zip':zipcode},
               **weather_info,
               **main_info,
               **wind_info,
               **loc_info}
        combined.append(sub)
    df = pd.DataFrame(combined)
    df.head()
    
    
    fig = px.scatter_mapbox(df,
                        lon = df['lon'],
                        lat = df['lat'],
                        hover_name = df['name'],
                        hover_data = df[['feels_like','temp_min','temp_max','humidity']],
                        zoom = 5,
                        color = df['temp'],
                        width = 900,
                        height = 600,
                        size = df['speed'],
                        color_continuous_scale="temps",
                        title = f'Weather Map of {county} county')
    fig.update_layout(mapbox_style = 'carto-positron')
    fig.show()

In [117]:
# get_weather_map()

In [118]:
# state = str(input("Please enter the USPS state code (ie. California is CA): "))
# stateinfo = State_info(state)
# dfstateinfo = pd.DataFrame(stateinfo['results'])

In [119]:
# dfstateinfo

In [120]:
# zipcodes = dfstateinfo['zip_code'][dfstateinfo['primary_coty_name'] == 'Los Angeles']
zipcodes = [95616]
combined = []
for zipcode in zipcodes:
    test_data = get_zip_weather(api_key,zipcode)
    #gather subseted data
    weather_info = test_data['weather'][0]
    main_info = test_data['main']
    wind_info = test_data['wind']
    loc_info = test_data['coord']
    city_info = test_data['name']
    # join all to a dictionary 
    sub = {**{'name':city_info},
            **{'zip':zipcode},
            **weather_info,
            **main_info,
            **wind_info,
            **loc_info}
    combined.append(sub)
df = pd.DataFrame(combined)
df.head()

,name,zip,id,main,description,icon,temp,feels_like,temp_min,temp_max,pressure,humidity,speed,deg,lon,lat
0,Davis,95616,800,Clear,clear sky,01d,60.42,58.28,57.16,62.98,1016,45,0,0,-121.7418,38.5538


### Historical and Forecast

In [121]:
# https://history.openweathermap.org/data/2.5/history/city?q=London,UK&appid={API key}

#
def get_hist_weather(api_key, city, country = 'US'):
    """Get 24 hour historical weather for city name."""
    weather_url = f"https://history.openweathermap.org/data/2.5/history/city?q={city},{country}&cnt={24}&appid={api_key}&units=imperial"
    response = requests.get(weather_url)
    return response.json()

def get_forecast_weather(api_key, city, country = 'US'):
    """Get 4 day hourly forecast for city name."""
    weather_url = f"https://pro.openweathermap.org/data/2.5/forecast/hourly?q={city},{country}&appid={api_key}&units=imperial"
    response = requests.get(weather_url)
    return response.json()

In [122]:
hist_data = get_hist_weather(api_key, 'Davis')

forecast_data = get_forecast_weather(api_key, 'Davis')

In [123]:
hist_data

{'message': 'Count: 24',
 'cod': '200',
 'city_id': 5341704,
 'calctime': 0.034816956,
 'cnt': 24,
 'list': [{'dt': 1702245600,
   'main': {'temp': 55.78,
    'feels_like': 53.4,
    'pressure': 1025,
    'humidity': 50,
    'temp_min': 53.65,
    'temp_max': 59},
   'wind': {'speed': 8.05, 'deg': 30},
   'clouds': {'all': 0},
   'weather': [{'id': 800,
     'main': 'Clear',
     'description': 'clear sky',
     'icon': '01d'}]},
  {'dt': 1702249200,
   'main': {'temp': 56.17,
    'feels_like': 53.74,
    'pressure': 1024,
    'humidity': 48,
    'temp_min': 54.88,
    'temp_max': 58.77},
   'wind': {'speed': 3.44, 'deg': 30},
   'clouds': {'all': 0},
   'weather': [{'id': 800,
     'main': 'Clear',
     'description': 'clear sky',
     'icon': '01d'}]},
  {'dt': 1702252800,
   'main': {'temp': 55.87,
    'feels_like': 53.46,
    'pressure': 1024,
    'humidity': 49,
    'temp_min': 53.87,
    'temp_max': 58.57},
   'wind': {'speed': 0, 'deg': 0},
   'clouds': {'all': 0},
   'weather':

In [124]:
# Build historical data frame
date_time = pd.DataFrame([pd.to_datetime(hist_data['list'][i]['dt'], unit='s') for i in range(len(hist_data['list']))])
main_data = pd.DataFrame([hist_data['list'][i]['main'] for i in range(len(hist_data['list']))])
wind_data = pd.DataFrame([hist_data['list'][i]['wind'] for i in range(len(hist_data['list']))])
cloud_data = pd.DataFrame([hist_data['list'][i]['clouds'] for i in range(len(hist_data['list']))])
weather_data = pd.DataFrame([hist_data['list'][i]['weather'][0] for i in range(len(hist_data['list']))])

hist_df = pd.concat([date_time, main_data, wind_data, cloud_data, weather_data], axis=1)
hist_df['data_type'] = "Historical"
hist_df

,0,temp,feels_like,pressure,humidity,temp_min,temp_max,speed,deg,all,id,main,description,icon,data_type
0,2023-12-10 22:00:00,55.78,53.40,1025,50,53.65,59.00,8.05,30,0,800,Clear,clear sky,01d,Historical
1,2023-12-10 23:00:00,56.17,53.74,1024,48,54.88,58.77,3.44,30,0,800,Clear,clear sky,01d,Historical
2,2023-12-11 00:00:00,55.87,53.46,1024,49,53.87,58.57,0.00,0,0,800,Clear,clear sky,01d,Historical
3,2023-12-11 01:00:00,52.02,49.60,1024,57,49.89,54.32,0.00,0,0,800,Clear,clear sky,01n,Historical
4,2023-12-11 02:00:00,48.25,48.25,1023,67,45.14,50.76,0.00,0,0,800,Clear,clear sky,01n,Historical
5,2023-12-11 03:00:00,46.06,46.06,1023,71,44.15,48.97,0.00,0,0,800,Clear,clear sky,01n,Historical
6,2023-12-11 04:00:00,45.59,45.59,1023,69,42.69,47.77,0.00,0,0,800,Clear,clear sky,01n,Historical
7,2023-12-11 05:00:00,42.66,42.66,1023,80,39.20,46.76,0.00,0,0,800,Clear,clear sky,01n,Historical
8,2023-12-11 06:00:00,40.46,40.46,1023,86,37.40,43.99,0.00,0,0,800,Clear,clear sky,01n,Historical
9,2023-12-11 07:00:00,39.54,39.54,1022,90,37.29,42.98,0.00,0,0,800,Clear,clear sky,01n,Historical


In [125]:
forecast_data

{'cod': '200',
 'message': 0,
 'cnt': 96,
 'list': [{'dt': 1702339200,
   'main': {'temp': 40.21,
    'feels_like': 40.21,
    'temp_min': 37.45,
    'temp_max': 40.21,
    'pressure': 1017,
    'sea_level': 1017,
    'grnd_level': 873,
    'humidity': 68,
    'temp_kf': 1.53},
   'weather': [{'id': 800,
     'main': 'Clear',
     'description': 'clear sky',
     'icon': '01n'}],
   'clouds': {'all': 0},
   'wind': {'speed': 1.95, 'deg': 353, 'gust': 2.93},
   'visibility': 10000,
   'pop': 0,
   'sys': {'pod': 'n'},
   'dt_txt': '2023-12-12 00:00:00'},
  {'dt': 1702342800,
   'main': {'temp': 39.38,
    'feels_like': 39.38,
    'temp_min': 36.05,
    'temp_max': 39.38,
    'pressure': 1018,
    'sea_level': 1018,
    'grnd_level': 873,
    'humidity': 67,
    'temp_kf': 1.85},
   'weather': [{'id': 800,
     'main': 'Clear',
     'description': 'clear sky',
     'icon': '01n'}],
   'clouds': {'all': 7},
   'wind': {'speed': 2.1, 'deg': 37, 'gust': 3.4},
   'visibility': 10000,
   'pop

In [126]:
# Build forecast data frame
date_time = pd.DataFrame([pd.to_datetime(forecast_data['list'][i]['dt'], unit='s') for i in range(len(forecast_data['list']))])
main_data = pd.DataFrame([forecast_data['list'][i]['main'] for i in range(len(forecast_data['list']))])
wind_data = pd.DataFrame([forecast_data['list'][i]['wind'] for i in range(len(forecast_data['list']))])
cloud_data = pd.DataFrame([forecast_data['list'][i]['clouds'] for i in range(len(forecast_data['list']))])
weather_data = pd.DataFrame([forecast_data['list'][i]['weather'][0] for i in range(len(forecast_data['list']))])


forecast_df = pd.concat([date_time, main_data, wind_data, cloud_data, weather_data], axis=1)
forecast_df['data_type'] = "Forecast"
forecast_df

,0,temp,feels_like,temp_min,temp_max,pressure,sea_level,grnd_level,humidity,temp_kf,speed,deg,gust,all,id,main,description,icon,data_type
0,2023-12-12 00:00:00,40.21,40.21,37.45,40.21,1017,1017,873,68,1.53,1.95,353,2.93,0,800,Clear,clear sky,01n,Forecast
1,2023-12-12 01:00:00,39.38,39.38,36.05,39.38,1018,1018,873,67,1.85,2.10,37,3.40,7,800,Clear,clear sky,01n,Forecast
2,2023-12-12 02:00:00,38.30,38.30,35.42,38.30,1019,1019,873,67,1.60,2.53,64,3.38,8,800,Clear,clear sky,01n,Forecast
3,2023-12-12 03:00:00,37.04,37.04,34.93,37.04,1019,1019,873,67,1.17,2.73,82,3.15,10,800,Clear,clear sky,01n,Forecast
4,2023-12-12 04:00:00,35.69,33.21,34.56,35.69,1020,1020,873,67,0.63,3.22,88,3.38,15,801,Clouds,few clouds,02n,Forecast
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,2023-12-15 19:00:00,41.86,41.86,41.86,41.86,1032,1032,885,48,0.00,2.01,39,2.26,10,800,Clear,clear sky,01d,Forecast
92,2023-12-15 20:00:00,42.98,42.98,42.98,42.98,1031,1031,884,47,0.00,1.39,20,2.04,11,801,Clouds,few clouds,02d,Forecast
93,2023-12-15 21:00:00,43.68,43.68,43.68,43.68,1031,1031,884,47,0.00,1.88,27,2.91,11,801,Clouds,few clouds,02d,Forecast
94,2023-12-15 22:00:00,43.81,43.81,43.81,43.81,1030,1030,883,49,0.00,1.63,25,2.71,31,802,Clouds,scattered clouds,03d,Forecast


In [127]:
#merge data for plotting
plot_df = pd.concat([hist_df, forecast_df], ignore_index=True, sort=True, axis=0)

In [128]:

fig = px.line(plot_df, x=0, y="temp", color="data_type")
# fig.add_scatter(plot_df, x=0, y="temp_min", color="data_type")

fig.show()

In [129]:
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Scatter(x=plot_df[0], y=plot_df["temp"],
                    line_shape='linear'))

In [130]:
#TODO: Plot historical and forecast data on same plot. Include lines for temp, max temp, min temp. Include hover option for other measures like wind speed, weather description, feels like.